# **Multilingual Translation with M2M100 and Prompt Engineering**

## **Introduction**

**M2M100** is a sequence-to-sequence model developed by Facebook AI, designed specifically to handle multilingual translation tasks. Unlike traditional models that are often trained to translate between specific pairs of languages, M2M100 is trained to translate directly between any pair of 100 languages. This makes it a perfect choice for building a translation system that supports multiple languages, as it can handle translation between any two of the supported languages without needing intermediary translations.



## **What is Prompt Engineering?**

**Prompt Engineering** is the process of designing and optimizing input prompts to get better, more accurate, and context-aware outputs from AI models, especially large language models (LLMs) like GPT, mBART,M2M100 and BERT.


## **Why Use Prompt Engineering in Language Translation?**
🚀 Since M2M100 is a powerful pre-trained multilingual model, it can still sometimes produce imperfect translations, particularly when handling complex or context-heavy sentences. Prompt engineering helps by:

✅ **Improving translation accuracy** by structuring input prompts in a way that the model understands better.

✅ **Enhancing fluency** by including example translations (few-shot prompting) to guide the model.

✅ **Explicit instructions** that provide clear guidelines, ensuring better results during translation.



In [84]:
# Install necessary libraries
!pip install indic-nlp-library nltk transformers sentencepiece

  Using cached indic_nlp_library-0.92-py3-none-any.whl.metadata (5.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 12.0 MB/s eta 0:00:00


# Import Required Libraries

In [94]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import nltk

* **PyTorch** - Used for tensor operations and running deep learning models efficiently.
* **M2M100Tokenizer** - Tokenizes input text and converts it into numerical representations that the model can process.
* **M2M100ForConditionalGeneration** -  A multilingual transformer-based model from Facebook’s M2M-100 family, designed for high-quality sequence-to-sequence tasks like translation.


In [77]:
# Download NLTK Punkt tokenizer models (if not already installed)
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Load the Pre-trained M2M100 Model and Tokenizer

In [95]:
model = "facebook/m2m100_418M"
tokenizer = M2M100Tokenizer.from_pretrained(model)
model = M2M100ForConditionalGeneration.from_pretrained(model)


## Defining Few-Shot Examples for Translation

In [96]:

few_shot_examples = {
    ("en_XX", "mr_IN"): [
        "Hello, how are you? -> नमस्कार, आपण कसे आहात?",
        "Good morning! -> शुभ प्रभात!",
        "What is your name? -> तुमचे नाव काय आहे?",
        "My name is John -> माझं नाव जॉन आहे.",
        "My name is John. Do not translate the name. -> माझं नाव जॉन आहे. नावाचे भाषांतर करू नका.",
        "My name is Seema. Do not translate the name. -> माझं नाव सीमा आहे. नावाचे भाषांतर करू नका."
    ],
    ("hi_IN", "en_XX"): [
        "नमस्कार, आप कैसे हैं? -> Hello, how are you?",
        "मुझे पढ़ाई पसंद है। -> I like studying.",
        "आपका नाम क्या है? -> What is your name?",
        "मेरा नाम सीमा है। -> My name is Seema.",
    ],
    ("mr_IN", "en_XX"): [
        "तुमचे नाव काय आहे? -> What is your name?",
        "माझे नाव रोहन आहे। -> My name is Rohan.",
        "आज हवामान छान आहे। -> The weather is nice today.",
        "माझे नाव सीमा आहे। -> My name is Seema.",
    ]
}

# Function for Translation with Refined Prompt

In [109]:
# Function for translation with refined prompt
def translate(text, source_lang, target_lang):
    """Translate text while avoiding unwanted instructions in the output."""
    # Retrieve few-shot examples if available
    prompt_examples = few_shot_examples.get((source_lang, target_lang), [])

    # Refined prompt with examples
    prompt_text = "\n".join(prompt_examples) + f"\n{text} "

    # Set source language
    tokenizer.src_lang = source_lang

    # Tokenize the input text and convert it into tensors
    model_inputs = tokenizer(prompt_text, return_tensors="pt")

    # Generate translated text
    translated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang]  # Set target language
    )

    # Decode the translated output
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

    # Remove the extra parts of the prompt, only keep the translation
    translated_text = translated_text.split("\n")[-1].strip()

    return translated_text

# Function to Translate a Paragraph


In [104]:
# Function to translate a paragraph
def translate_paragraph(paragraph, source_lang, target_lang):
    """Translate the entire paragraph by first breaking it into sentences."""

    # Break the paragraph into sentences using NLTK's Punkt tokenizer
    sentences = nltk.sent_tokenize(paragraph)

    # Translate each sentence and collect the translations
    translated_sentences = []
    for sentence in sentences:
        translated_sentence = translate(sentence, source_lang, target_lang)
        translated_sentences.append(translated_sentence)

    # Join the translated sentences to form a paragraph
    translated_paragraph = ' '.join(translated_sentences)

    return translated_paragraph


# Example

In [110]:
# Example Hindi to English translation
text1 = "मुझे यात्रा करना पसंद है।"
translated_text3 = translate(text1, "hi", "en")
print(f"Hindi to English:\n{text1} -> {translated_text3}")


Hindi to English:
मुझे यात्रा करना पसंद है। -> I like to travel.


In [111]:
# Paragraph translation (Marathi to English)
paragraph_mr = """नाव सोपे, लहान आणि अर्थपूर्ण असावे. उच्चारणासाठी कठीण, लांब किंवा खिल्ली उडवली जाणारी नावे टाळावीत. प्राचीन किंवा दुर्मिळ नावांच्या ऐवजी, काळास अनुसरून असलेले आणि कॉमन नसलेले नाव निवडावे. नावाचा अर्थ सकारात्मक आणि प्रेरणादायी असावा जेणेकरून बाळाचे व्यक्तिमत्त्व चांगले विकसीत होईल."""
translated_paragraph_mr = translate_paragraph(paragraph_mr, "mr", "en")
print(f"Translated Paragraph (Marathi to English):\n{translated_paragraph_mr}")


Translated Paragraph (Marathi to English):
The name is simple, small and meaningful. Difficult, long or flushed for expression. The name of the ancient or ancient names, followed by the black and uncommon names. The name means positive and motivating rather than that the baby's personality will develop well.


In [112]:
# Paragraph translation (English to Marathi)
paragraph_en = """The Sun is the only star in our solar system. It is the center of our solar system, and its gravity holds the solar system together. Everything in our solar system revolves around it – the planets, asteroids, comets, and tiny bits of space debris."""
translated_paragraph_en = translate_paragraph(paragraph_en, "en", "mr")
print(f"Translated Paragraph (English to Marathi):\n{translated_paragraph_en}")


Translated Paragraph (English to Marathi):
सूर्य आपल्या सौर प्रणालीत एकमात्र तारा आहे. हे आपल्या सौर प्रणालीचे केंद्र आहे आणि त्याचे गुरुत्वाकर्षण सौर प्रणाली एकत्र ठेवतो. आपल्या सौर प्रणालीतील सर्व गोष्टी त्यावर घरी जातात – ग्रह, एस्टेरॉयड, कॉमेट, आणि अंतरिक्षचे छोट्या बिट.
